# Build an Interactice Finance Agent

> IMPORTANT: This is the fifth and final notebook in the lab. The notebooks build on top of each other, so be sure to run the preceding notebooks, in order, before running this one. Start your journey building ADK Agents with MCP Toolbox [here](./1_setup_and_explore_databases.ipynb). 

## Overview 

Welcome to the final notebook of the lab! This is where you will bring everything you've built together to create a fully interactive, enterprise-ready Fraud Analyst Agent. This notebook guides you through deploying advanced MCP Toolbox tools, backed by Spanner Graph and AlloyDB Natural Language, and a user-friendly web interface that allows a fraud analyst to interact with the powerful AI agent you've developed. The key accomplishments include:

- Building an Interactive UI: You will use the ADK UI library to create a simple yet powerful web interface for your agent. This UI provides a chat-like experience, making it easy for non-technical users to securely ask complex questions about the financial data in the Spanner Graph and AlloyDB databases.
- Deploying a Secure Web Application: You will deploy the interactive UI as a new Cloud Run service. Crucially, this deployment will be secured with Identity-Aware Proxy (IAP), ensuring that only authorized users, such as verified fraud analysts, can access the application.
- Integrating All Components: This final step demonstrates how the ADK Agent, MCP Toolbox, AlloyDB, Spanner, and the new UI all work together in a secure, private network to create a cohesive and powerful solution.
- End-to-End Fraud Investigation: You will use the final application to perform a sample fraud investigation, asking the agent a series of questions to identify suspicious activity. This showcases the real-world value of having an AI assistant that can instantly query multiple, complex databases.

### Terraform Resources

- Google Cloud APIs: The necessary APIs for services like Vertex AI, Cloud Run, Spanner, AlloyDB, and Secret Manager were all enabled to allow these services to work together.
- VPC Network and Connectivity: A custom Virtual Private Cloud (VPC) named demo-vpc was created, along with a Cloud Router and Cloud NAT gateway, to provide a secure private network for your resources.
- Firewall Rules: Firewall rules, such as allow-iap-internal, were configured to control traffic flow within the VPC, including allowing necessary communication for IAP.
- Private IP Allocation: A private IP range was reserved for service networking, enabling services like AlloyDB and Cloud SQL to communicate privately within your VPC.
- AlloyDB Cluster and Instance: A high-performance AlloyDB cluster and instance were provisioned to store and manage your financial data.
- Cloud SQL for PostgreSQL Instance: A Cloud SQL instance was created to serve as a persistent session backend for your ADK agent, ensuring that conversation history is maintained.
- Spanner Instance and Database: A Spanner instance and database were set up with a predefined schema, including a property graph, to handle complex relational and graph-based queries.
- Vertex AI Workbench Instance: The Vertex AI Workbench instance you are currently using was created and configured with the necessary permissions and startup scripts to access the project's resources.
- IAM Service Accounts: The toolbox-service-account service account was created with the specific IAM roles required to access databases and secrets securely.
- Cloud Storage Bucket: A Cloud Storage bucket was created to store project files, including the data for the Spanner database import.
- Private DNS Zone: A private DNS zone was configured to enable PSC (Private Service Connect) for secure communication with the Cloud SQL instance.

### Google Cloud Services Used

This notebook utilizes the following Google Cloud services:

- Agent Development Kit (ADK): The core framework for building and deploying your interactive financial agent.
- MCP Toolbox for Databases: The open-source server that provides the tools for your agent to interact with the databases.
- Cloud Run: The serverless platform used to deploy and host both the ADK agent and the MCP Toolbox.
- AlloyDB for PostgreSQL: A fully managed, PostgreSQL-compatible database used to store financial transaction data.
- Spanner: A globally distributed, strongly consistent database used for both relational and graph-based financial data.
- Identity-Aware Proxy (IAP): A service that provides secure, identity-based access to your deployed web applications, including the ADK agent's UI.
- Secret Manager: A secure service for storing and managing sensitive data like API keys, passwords, and configuration files.
- Cloud SQL for PostgreSQL: A managed PostgreSQL database service used for the ADK agent's session management.
- IAM (Identity and Access Management): Used to manage permissions for service accounts and users, ensuring a least-privilege security model.
- Vertex AI Workbench: The Jupyter notebook-based environment used to develop and execute the code in this lab.
- Cloud Storage: Used to store the data files for the initial database setup.
- Cloud Logging and Cloud Trace: These services are used for monitoring, debugging, and observing the behavior of your deployed services.

### Logical Flow of the Notebook

The notebook will guide you through the following logical steps:
- Basic Setup: This initial section involves defining the necessary variables for your project, connecting to your Google Cloud environment, and installing the required Python libraries.
- Configure a New Toolset: You will create an updated tools.yaml file that defines a more powerful and flexible toolset for your agent. This toolset will include tools for natural language queries, graph queries, account status checks, and more.
- Deploy the Interactive Agent: You will deploy the ADK agent to Cloud Run, this time configuring it for public access secured by IAP. This will enable you to interact with the agent through its web UI.
- Grant IAP Access: You will grant the necessary IAM permissions to service accounts and your user account to allow access to the IAP-secured web UI.
- Test the Agent: Using the ADK's built-in web UI, you will engage in a series of interactive scenarios with your agent, asking it to perform tasks related to fraud investigation and financial analysis.

## Basic Setup

### Define Notebook Variables

Update the `project_id` and `region` variables below to match your environment. You can use defaults for the rest of the project variables. 

You will be prompted for the Cloud SQL password you chose then you provisioned the environment with Terraform.

In [ ]:
# Project variables
project_id = "your-project"
region = "your-region"
vpc = "demo-vpc"
gcs_bucket_name = f"project-files-{project_id}"

# AlloyDB variables
alloydb_cluster = "my-alloydb-cluster"
alloydb_instance = "my-alloydb-instance"
alloydb_database = "finance"

# Spanner variables
spanner_instance = "my-spanner-instance"
spanner_database = "finance-graph"

# Cloud SQL Variables
cloud_sql_instance = "my-postgres-instance"
cloud_sql_database = "postgres"
cloud_sql_user = "postgres"
cloud_sql_password = input("Please enter the password for the Cloud SQL 'postgres' database user: ")

In [ ]:
# Set env variable to suppress annoying system warnings when running shell commands
%env GRPC_ENABLE_FORK_SUPPORT=1

### Connect to your Google Cloud Project

In [ ]:
# Configure gcloud.
!gcloud config set project {project_id}

### Configure Logging

In [ ]:
import logging
import sys

# Configure the root logger to output messages with INFO level or above
logging.basicConfig(level=logging.INFO, stream=sys.stdout, format='%(asctime)s[%(levelname)5s][%(name)14s] - %(message)s',  datefmt='%H:%M:%S', force=True)

### Install Dependencies

In [ ]:
! pip install --quiet toolbox-core==0.5.2 \
                      google-adk==1.5.0 \
                      google-genai==1.23.0 \
                      asyncpg==0.30.0 \
                      "cloud-sql-python-connector[asyncpg]"==1.18.2 \
                      google-cloud-iam==2.19.0 \
                      google-cloud-iap==1.17.1 \
                      google-cloud-resource-manager==1.14.2


### Define Helper Functions

#### get_auth_token()

In [ ]:
import urllib

import google.auth.transport.requests
import google.oauth2.id_token


def get_auth_token(audience):
    # For IAM auth, Cloud Run uses your service's hostname as the `audience` value
    # i.e. audience = 'https://my-cloud-run-service.run.app/'
    
    # For IAP auth, Cloud Run uses the service account's oauth client_id
    # i.e. audience = '123456789098765432123'
    
    auth_req = google.auth.transport.requests.Request()
    id_token = google.oauth2.id_token.fetch_id_token(auth_req, f"{audience}")

    return id_token

#### get_project_number()

In [ ]:
from google.auth.transport.requests import Request
from google.cloud import resourcemanager_v3


def get_project_number(project_id: str) -> str:
    # Create a client for the Resource Manager API.
    client = resourcemanager_v3.ProjectsClient()

    # Construct the project name resource string.
    project_name = f"projects/{project_id}"

    # Create the request to get the project details.
    request = resourcemanager_v3.GetProjectRequest(name=project_name)

    # Make the API call to retrieve the project.
    project = client.get_project(request=request)

    # The project number is a string, not an integer.
    return project.name.split('/')[1]


# Configure Cloud Run Services with Identity-Aware Proxy (IAP)

We previously restricted non-private ingress traffic on our ADK Agent hosted on Cloud Run. This is a great security practice for autonomous agents that are triggered by an event and take automated actions on your behalf. For interactive agents, however, it is useful to provide a friendly UI for Agent interactions, and the ADK framework provides a built-in UI specifically for this purpose. To make this UI available without sacrificing our security posture, we will replace IAM authentication with IAP to prevent unauthorized access to the Agent, while allowing authorized individuals to safely connect over public IP. We will also configure IAP on the toolbox service, and we'll use the toolbox-service-account oauth client ID to authenticate calls between the ADK agent and MCP Toolbox.

Reference: 
- https://cloud.google.com/iap/docs/enabling-cloud-run

## Update the MCP Toolbox Toolset

The goal of our interactive agent is to be as flexible as possible, so we will use an alloydb_nl tool to flexibly and securely interact with our AlloyDB database. We will also build tools that will allow us to ask questions about account transfers that might be related to fraudulent transactions and to block or unblock accounts. 

### Update the `tools.yaml` File

This YAML configuration defines the tools our agent will use. Notice the use of both 'spanner-sql' for precise queries and 'alloydb-ai-nl' for flexible, natural language queries. Read through the definitions of each tool to understand the purpose of each query.

In [ ]:
# Reference: https://googleapis.github.io/genai-toolbox/resources/sources/spanner/
#            https://googleapis.github.io/genai-toolbox/resources/tools/
#            https://googleapis.github.io/genai-toolbox/resources/tools/spanner-sql/
#            https://googleapis.github.io/genai-toolbox/resources/sources/alloydb-pg/
#            https://googleapis.github.io/genai-toolbox/resources/tools/postgres-sql/

import os
import json

nl_config = "finance_agent_config"

tools_config = {
  "sources": {
    "spanner-finance-graph-source": {
      "kind": "spanner",
      "project": f"{project_id}",
      "instance": f"{spanner_instance}",
      "database": f"{spanner_database}",
      "dialect": "googlesql"
    },
    "alloydb-finance-source": {
      "kind": "alloydb-postgres",
      "project": f"{project_id}",
      "region": f"{region}",
      "cluster": f"{alloydb_cluster}",
      "instance": f"{alloydb_instance}",
      "database": f"{alloydb_database}",
      "user": "toolbox_user",
      "password": "${ALLOYDB_PASSWORD}",
      "ipType": "private"
    }
  },
  "tools": {
    "get_account_status": {
      "kind": "spanner-sql",
      "source": "spanner-finance-graph-source",
      "description": "Use this tool to get the status of an account based on its ID, including blocked status, account type, and account owner name.",
      "statement": """
        SELECT * FROM Account a
        JOIN PersonOwnAccount poa ON a.id = poa.account_id
        JOIN Person p on p.id = poa.id
        WHERE a.id = @account_id;
        """,
      "parameters": [
        {
          "name": "account_id",
          "type": "integer",
          "description": "Unique account id number"
        }
      ]
    },
    "block_account": {
      "kind": "spanner-sql",
      "source": "spanner-finance-graph-source",
      "description": "Use this tool to block an account based on its ID. The query will block the account and then return the updated blocked status and type of the account.",
      "statement": """
        UPDATE Account SET is_blocked = true WHERE id = @account_id
        THEN RETURN id, is_blocked, type;
        """,
      "parameters": [
        {
          "name": "account_id",
          "type": "integer",
          "description": "Unique account id number"
        }
      ]
    },
    "unblock_account": {
      "kind": "spanner-sql",
      "source": "spanner-finance-graph-source",
      "description": "Use this tool to unblock an account based on its ID. The query will unblock the account and then return the updated blocked status and type of the account.",
      "statement": """
        UPDATE Account SET is_blocked = false WHERE id = @account_id
        THEN RETURN id, is_blocked, type;
        """,
      "parameters": [
        {
          "name": "account_id",
          "type": "integer",
          "description": "Unique account id number"
        }
      ]
    },
    "get_accounts_by_customer_id": {
      "kind": "spanner-sql",
      "source": "spanner-finance-graph-source",
      "description": "Use this tool to get all of the accounts owned by a particular customer based on their id",
      "statement": """
        SELECT * FROM Account a
        JOIN PersonOwnAccount poa ON a.id = poa.account_id
        JOIN Person p on p.id = poa.id
        WHERE p.id = @customer_id;
        """,
      "parameters": [
        {
          "name": "customer_id",
          "type": "integer",
          "description": "Unique customer id number"
        }
      ]
    },
    "get_loan_repayments": {
      "kind": "spanner-sql",
      "source": "spanner-finance-graph-source",
      "description": "Use this tool to get information about loans paid by customers from accounts they own. The query finds the accounts owned by the customer, and then discover the loans repaid by their accounts",
      "statement": """
        GRAPH FinGraph
        MATCH
          (person:Person {id: @customer_id})-[own:Owns]->
          (account:Account)-[repay:Repays]->(loan:Loan)
        RETURN
          person.id AS customer_id,
          person.name AS customer_name,
          account.id AS account_id,
          repay.create_time AS repay_time,
          repay.amount AS loan_repay_amount,
          loan.id AS loan_id,
          loan.loan_amount AS loan_amount,
          SAFE_TO_JSON(person) AS person_json, 
          SAFE_TO_JSON(own) AS own_json, 
          SAFE_TO_JSON(account) AS account_json, 
          SAFE_TO_JSON(repay) AS repay_json, 
          SAFE_TO_JSON(loan) AS loan_json
        ORDER BY repay.create_time DESC;
        """,
      "parameters": [
        {
          "name": "customer_id",
          "type": "integer",
          "description": "Unique customer id number"
        }
      ]
    },
    "get_indirect_transfers": {
      "kind": "spanner-sql",
      "source": "spanner-finance-graph-source",
      "description": "Use this tool to check if money was moved between multiple accounts on its way from a given source account to a given destination account. The query matches all account money transfers starting from a source account id within 3 to 6 hops, to reach a destination account with another id. The {3,6} syntax is used to represent a quantified 3 to 6 hop path traversal between src_accnt and dst_accnt.",
      "statement": """
        GRAPH FinGraph
        MATCH
          (src_accnt:Account {id:@source_account_id})-[transfers:Transfers]->{3,6}
          (dst_accnt:Account {id:@destination_account_id})
        RETURN
          ARRAY_LENGTH(transfers) AS num_hops,
          src_accnt.type as source_account_type,
          src_accnt.is_blocked AS source_is_blocked,
          src_accnt.create_time AS source_create_time,
          dst_accnt.type as destination_account_type,
          dst_accnt.is_blocked AS destination_is_blocked,
          dst_accnt.create_time AS destinatoin_create_time,
          TO_JSON(transfers) AS transfer_edges;
        """,
      "parameters": [
        {
          "name": "source_account_id",
          "type": "integer",
          "description": "Unique account id number for source account"
        },
        {
          "name": "destination_account_id",
          "type": "integer",
          "description": "Unique account id number for destination account"
        }
      ]
    },
    "get_account_inflows_outflows": {
      "kind": "spanner-sql",
      "source": "spanner-finance-graph-source",
      "description": "Use this tool to understand the money flow of accounts owned by a specific customer id to look for suspicious large inflows or outflows as part of compliance analysis requirements.",
      "statement": """
        GRAPH FinGraph
        MATCH (person:Person {id: @customer_id})-[:Owns]->(accnt:Account)
        RETURN person, accnt

        NEXT

        MATCH (accnt)<-[inflow:Transfers]-(:Account)
        WHERE inflow.amount > 100
          AND inflow.create_time > TIMESTAMP("2020-1-1")
        RETURN person, accnt, SUM(inflow.amount) AS total_inflow_amounts
        GROUP BY person, accnt

        NEXT

        MATCH (accnt) -[outflow:Transfers]->(:Account)
        WHERE outflow.amount > 100
          AND outflow.create_time > TIMESTAMP("2020-1-1")
        RETURN person, accnt, total_inflow_amounts, SUM(outflow.amount) AS total_outflow_amounts
        GROUP BY person, accnt, total_inflow_amounts

        NEXT

        LET money_flow_ratio = total_inflow_amounts / total_outflow_amounts
        RETURN 
          person.name AS customer_name,
          person.id AS customer_id,
          accnt.id AS account_id, 
          accnt.type AS account_type,
          accnt.is_blocked,
          total_inflow_amounts, 
          total_outflow_amounts,
          money_flow_ratio
        ORDER BY money_flow_ratio DESC;
        """,
      "parameters": [
        {
          "name": "customer_id",
          "type": "integer",
          "description": "Unique customer id number"
        }
      ]
    },
    "get_destination_account_audit_events": {
      "kind": "spanner-sql",
      "source": "spanner-finance-graph-source",
      "description": "Use this tool to find audit events of reached destination accounts that have received wire transfers from a suspicious account.",
      "statement": """
        SELECT
          audit.id as accnt_id,
          audit.audit_timestamp as audit_ts,
          audit.audit_details as details,
          accnt_transfers.*
        FROM
          AccountAudits audit,
          GRAPH_TABLE(
            FinGraph
            MATCH (suspicious_account:Account {id:@suspicious_account_id})-[:Transfers]->{1,2}(dest_accnt)
            RETURN DISTINCT dest_accnt.id AS reached_account_id, 
              dest_accnt.type AS reached_account_type, 
              dest_accnt.is_blocked AS reached_account_is_blocked, 
              dest_accnt.create_time AS reached_account_create_time,
              suspicious_account.type AS suspicious_account_type, 
              suspicious_account.is_blocked AS suspicious_account_is_blocked, 
              suspicious_account.create_time AS suspicious_account_create_time
          ) AS accnt_transfers
        WHERE accnt_transfers.reached_account_id = audit.id
        ORDER BY audit_ts DESC
        """,
      "parameters": [
        {
          "name": "suspicious_account_id",
          "type": "integer",
          "description": "Unique account id number for an account with suspicious activity"
        }
      ]
    },
    "get_finance_database_context": {
      "kind": "alloydb-ai-nl",
      "source": "alloydb-finance-source",
      "description": "Use this tool to look up information about financial transactions, credit cards, customers, mcc codes, and historical fraud labels.",
      "nlConfig": f"{nl_config}"
    }
  },
  "toolsets": {
    "interactive-tools": [
      "get_loan_repayments",
      "get_indirect_transfers",
      "get_account_inflows_outflows",
      "get_destination_account_audit_events",
      "get_finance_database_context",
      "get_account_status",
      "get_accounts_by_customer_id",
      "block_account",
      "unblock_account"
    ]
  }
}

with open("tools.yaml", "w") as file:
    file.write(json.dumps(tools_config))


### Write Updated `tools.yaml` to Secret Manager

We store the tools.yaml file in Secret Manager to avoid exposing it in our code or container. This is a critical security best practice for managing sensitive configurations.

In [ ]:
# Create the secret
! gcloud secrets versions add tools --data-file=tools.yaml

In [ ]:
# Clean up the local file
import os
os.remove('tools.yaml')

### Redeploy Toolbox with IAP

Here, we redeploy the MCP Toolbox service with IAP enabled. 

Key security configurations:
- `--network={vpc}` / `--subnet={vpc}`: Deploys the service within our private VPC.
- `--service-account=toolbox-service-account`: Uses a dedicated service account with least-privilege permissions.
- `--no-allow-unauthenticated`: Enforces IAM authentication for all invocations.
- `--set-secrets`: Securely mounts our tools configuration and database password from Secret Manager.
- `--ingress=internal`: Restricts network traffic to internal sources only.
- `--telemetry-gcp`: Enables built-in OpenTelemetry for logging, tracing, and metrics.
- `--iap`:  Secures the service, allowing access only to authenticated and authorized users.


In [ ]:
# Define Toolbox Container Image
image = f'{region}-docker.pkg.dev/database-toolbox/toolbox/toolbox:latest'

# Deploy to Cloud Run
! gcloud beta run deploy toolbox --no-user-output-enabled \
    --image={image} \
    --network={vpc} \
    --subnet={vpc} \
    --region={region} \
    --no-allow-unauthenticated \
    --set-secrets="/app/tools.yaml=tools:latest,ALLOYDB_PASSWORD=alloydb-password:latest" \
    --args="--tools-file=/app/tools.yaml","--address=0.0.0.0","--port=8080","--telemetry-gcp" \
    --vpc-egress=all-traffic \
    --ingress=internal \
    --min=1 \
    --service-account=toolbox-service-account \
    --iap


## Grant Necessary Service Account Permissions for the Toolbox Service

Now that we are using both IAP and IAM to protect the toolbox service, we need to grant permissions in 3 places for this to work:
1. IAM `roles/run.invoker` permissions on the toolbox service for the **client service account** (e.g. the notebook service account).
2. IAP `roles/iap.httpsResourceAccessor`role policy permissions on the toolbox service for the **client service account** (e.g. the notebook service account).
3. IAP programmatic access on the project for the oauth client ID of the **service-side service account** (e.g. `toolbox-service-account`). 

### Grant `roles/run.invoker` to the Notebook Service Account

We still enabled IAM authentication on the toolbox service as a second layer of security, so we will grant invoke access to notebook service account first so that we can test locally. 

In [ ]:
# Get the notebook service account from the local metadata service
notebook_service_account = !curl -s "http://metadata.google.internal/computeMetadata/v1/instance/service-accounts/default/email" -H "Metadata-Flavor: Google"
notebook_service_account = notebook_service_account[0]
print(f"Notebook service account: {notebook_service_account}")

# Grant IAM invoke permissions on toolbox to the notebook service account
!gcloud run services add-iam-policy-binding toolbox --region={region} \
    --member=serviceAccount:{notebook_service_account} \
    --role="roles/run.invoker"

### Grant `roles/iap.httpsResourceAccessor` to the Notebook Service Account

In [ ]:
# Grant httpsResourceAccessor IAP access on toolbox to the notebook service account
!gcloud beta iap web add-iam-policy-binding \
    --member=serviceAccount:{notebook_service_account} \
    --role=roles/iap.httpsResourceAccessor \
    --region={region} \
    --resource-type=cloud-run \
    --service=toolbox \
    --condition=None

### Configure the Oauth Consent Screen

This manual step is mandatory for the following steps to work properly. Without it, you can't set programmatic access for the toolbox-service-account, and you can't login successfully through IAP. 

Pause now, and follow the steps [in this doc](https://developers.google.com/workspace/guides/configure-oauth-consent) to configure the consent screen. You can use values like the following:
- App name: `finance-agent`
- User support email: Select your student email from the drop-down.
- Audience: `Internal`
- Contact information: `test@example.com` (can be any email)
- Agree to the user policy and click "Create"

### Grant Programmatic IAP Access to `toolbox-service-account`

This step is CRITICAL for enabling service-to-service authentication. Without it, you will receive errors like the ones below when trying to execute MCP Toolbox Tools locally from this notebook or through the ADK agent that we will deploy later in this notebook: 
- `RuntimeError: API request failed with status 401 (Unauthorized). Server response: Invalid IAP credentials: Invalid bearer token. Invalid JWT audience.`
- `Attempt to decode JSON with unexpected mimetype: text/html`

References:
- https://cloud.google.com/iap/docs/authentication-howto#authenticate_with_an_oidc_token
- https://cloud.google.com/iap/docs/sharing-oauth-clients#gcloud

In [ ]:
# Get the toolbox-service-account oauth client id
toolbox_oauth_id = !gcloud iam service-accounts describe toolbox-service-account@{project_id}.iam.gserviceaccount.com --format='value("oauth2ClientId")'
toolbox_oauth_id = toolbox_oauth_id[0]
print(f"OAuth Client ID for toolbox-service-account: {toolbox_oauth_id}")

# Define programmatic clients for IAP
iap_programmatic_access_settings = f"""  access_settings:
    oauth_settings:
      programmatic_clients: '{toolbox_oauth_id}'
"""

# Write config to temp file
with open("iapConfig.yaml", "w") as file:
    file.write(iap_programmatic_access_settings)
    
# Apply the IAP programmatic client settings
!gcloud iap settings set iapConfig.yaml --project={project_id} --resource-type=iap_web

# Clean up the temp file
os.remove("iapConfig.yaml")

## Test the MCP Toolbox Tools Locally

Since we now have a new authentication scheme using IAM, as well as a more complex set of tools that require various parameters, we'll define a list of tools and the parameters we should call for each one, then we'll iterate through each one using the `load_tool()` function to get and test specific tools. 

> IMPORTANT: You must use the toolbox-service-account oauth client ID as the audience for your auth_token. If you try to use the toolbox_url for the auth_token audience like we did in previous notebooks, you will get an error like the following: `RuntimeError: API request failed with status 401 (Unauthorized). Server response: Invalid IAP credentials: Invalid bearer token. Audience doesn't match the allowlisted oauth clients for this application.`

> NOTE: If you get aa 403 error, wait a minute or two and try again. It takes time for the permissions you added in the cells above to propagate. Example error: `API request failed with status 403 (Forbidden). Server response: Access denied.`

In [ ]:
import json
import toolbox_core
from toolbox_core import ToolboxClient, auth_methods

# Get the toolbox-service-account oauth client id
toolbox_oauth_id = !gcloud iam service-accounts describe toolbox-service-account@{project_id}.iam.gserviceaccount.com --format='value("oauth2ClientId")'
toolbox_oauth_id = toolbox_oauth_id[0]
print(f"OAuth Client ID for toolbox-service-account: {toolbox_oauth_id}")

# Get toolbox endpoint
toolbox_url = ! gcloud run services describe toolbox --region {region} --format 'value(metadata.annotations."run.googleapis.com/urls")'
toolbox_url = json.loads(toolbox_url[0])[0]
print(f"Toolbox Cloud Run endpoint: {toolbox_url}")

# Refresh auth_token
print("Using toolbox-service-account for auth_token audience to support IAP authentication.")
auth_token = get_auth_token(toolbox_oauth_id)

# Define tools to test and arguments
tools_to_test = [
    {
        "tool": "get_loan_repayments",
        "params": {
            "customer_id": 263
        }
    },
    {
        "tool": "get_indirect_transfers",
        "params": {
            "source_account_id": 75,
            "destination_account_id": 199
        }
    },
    {
        "tool": "get_account_inflows_outflows",
        "params": {
            "customer_id": 330
        }
    },
    {
        "tool": "get_destination_account_audit_events",
        "params": {
            "suspicious_account_id": 11
        }
    },
    {
        "tool": "get_finance_database_context",
        "params": "How many users are in the database?"
    },
    {
        "tool": "get_account_status",
        "params": {
            "account_id": 75
        }
    },
    {
        "tool": "get_accounts_by_customer_id",
        "params": {
            "customer_id": 330
        }
    },
]

# Run tools 
async with ToolboxClient(
    toolbox_url,
    client_headers={"Authorization": f"Bearer {auth_token}"},
) as client:
    for t in tools_to_test:
        print(f"\n\nTesting tool: { t.get('tool') }, with Params: { t.get('params') }")
        tool = await client.load_tool(t.get('tool'))
        params = t.get('params')
        if isinstance(params, dict):
            result = await tool(**params)
        else:
            result = await tool(params)

        json_result = json.loads(result)
        print("\nTool result:")
        print(json.dumps(json_result, indent=2))
    


## Deploy the Interactive ADK Agent

This process is the same as the agent we configured in notebook 3, but this time we exposing the service to public IP traffic and protecting it with [Identity-Aware Proxy](https://cloud.google.com/iap/docs/enabling-cloud-run). We'll also update the agent to use the new `interactive-tools` toolset that we just deployed to MCP Toolbox, and we'll use the oauth client ID of the toolbox-service-account service account as the audience for the auth_token to allow the ASK agent to securely interact with MCP Toolbox.

References:
- https://github.com/alphinside/deploy-and-manage-adk-service/tree/main/weather_agent
- https://codelabs.developers.google.com/deploy-manage-observe-adk-cloud-run#1

### Use Cloud SQL for Session Persistence

The Agent Development Kit (ADK) uses a Session Service to maintain the state of conversations. By default, it uses an in-memory session service, but this is not suitable for production environments where the server may restart, and sessions would be lost. To ensure persistence, we can connect the ADK to a Cloud SQL database to act as the persistent store for the Session service.

### Create `__init__.py`

In [ ]:
# Initialize the agent directory
! mkdir -p deploy
! mkdir -p deploy/finance_agent

In [ ]:
%%writefile deploy/finance_agent/__init__.py
# __init__.py

from .agent import root_agent

__all__ = ["root_agent"]

### Create `agent.py`

In [ ]:
%%writefile deploy/finance_agent/agent.py
import os
import google.auth
import google.auth.transport.requests
import google.oauth2.id_token
from google.adk.agents import Agent
from toolbox_core import ToolboxSyncClient
from opentelemetry.instrumentation.aiohttp_client import (AioHttpClientInstrumentor)


def get_auth_token(audience):
    # For IAM auth, Cloud Run uses your service's hostname as the `audience` value
    # i.e. audience = 'https://my-cloud-run-service.run.app/'
    
    # For IAP auth, Cloud Run uses the service account's oauth client_id
    # i.e. audience = '123456789098765432123'
    
    auth_req = google.auth.transport.requests.Request()
    id_token = google.oauth2.id_token.fetch_id_token(auth_req, f"{audience}")

    return id_token


# The URL for the toolbox service and oauth id should be passed as environment variables
TOOLBOX_URL = os.getenv("TOOLBOX_URL")
if not TOOLBOX_URL:
    raise ValueError("TOOLBOX_URL environment variable not set.")
print(f"Using TOOLBOX_URL to connect agent to MCP Toolbox: {TOOLBOX_URL}")
    
TOOLBOX_OAUTH_CLIENT_ID = os.getenv("TOOLBOX_OAUTH_CLIENT_ID")
if not TOOLBOX_OAUTH_CLIENT_ID:
    raise ValueError("TOOLBOX_OAUTH_CLIENT_ID environment variable not set.")
print(f"Using TOOLBOX_OAUTH_CLIENT_ID url as oauth audience: {TOOLBOX_OAUTH_CLIENT_ID}")

# Get an auth token to invoke the Toolbox on Cloud Run
auth_token = get_auth_token(TOOLBOX_OAUTH_CLIENT_ID)

# Initialize the Toolbox client once and reuse it
print(f"Connecting to toolbox server at {TOOLBOX_URL}")
toolbox_client = ToolboxSyncClient(
    TOOLBOX_URL,
    client_headers={"Authorization": f"Bearer {auth_token}"}
)

# Define the agent's instructions
prompt = """
You're a helpful financial assistant. You handle fraud detection tasks, transaction lookups,
account details, loan payments, account transfers,suspicious activity reports, and other tasks 
related to financial data.
"""

# Define the root agent
root_agent = Agent(
    model='gemini-2.5-flash',
    name='interactive_finance_agent',
    description='A helpful AI assistant for financial tasks.',
    instruction=prompt,
    # Load the tools from your MCP Toolbox
    tools=toolbox_client.load_toolset("interactive-tools"),
)

# Enable the AioHttp library for Toolbox tracing
AioHttpClientInstrumentor().instrument()

### Create `server.py`

In [ ]:
%%writefile deploy/server.py
import os
from fastapi import FastAPI
from google.adk.cli.fast_api import get_fast_api_app
from google.cloud import logging as google_cloud_logging

# OTEL imports
from opentelemetry import trace
from opentelemetry.sdk.trace import TracerProvider, export
from opentelemetry.propagate import set_global_textmap
from opentelemetry.propagators.cloud_trace_propagator import CloudTraceFormatPropagator
from opentelemetry.exporter.cloud_trace import CloudTraceSpanExporter

# Initialize the Google Cloud Logging client
logging_client = google_cloud_logging.Client()
logger = logging_client.logger(__name__)

# The directory where your agent.py and __init__.py are located
AGENT_DIR = os.path.dirname(os.path.abspath(__file__))

# Get the session service URI from environment variables.
# This is where you'll provide the connection string for Cloud SQL.
session_uri = os.getenv("SESSION_SERVICE_URI")

# Prepare arguments for the FastAPI app
app_args = {"agents_dir": AGENT_DIR, "web": True}

# Use the Cloud SQL session service if the URI is provided
if session_uri:
    app_args["session_service_uri"] = session_uri
    logger.log_text(f"Using database session service.", severity="INFO")
else:
    logger.log_text(
        "SESSION_SERVICE_URI not provided. Using in-memory session service. "
        "Sessions will be lost on server restart.",
        severity="WARNING",
    )
    
# Observability
provider = TracerProvider()
processor = export.BatchSpanProcessor(CloudTraceSpanExporter())
provider.add_span_processor(processor)
trace.set_tracer_provider(provider)
print("Set trace provider")

# Create the FastAPI app using the ADK utility
app: FastAPI = get_fast_api_app(**app_args)

app.title = "finance-agent"
app.description = "API for interacting with the ADK Finance Agent"

# Main execution
if __name__ == "__main__":
    import uvicorn
    
    # Enable the AioHttp library for Toolbox tracing
    AioHttpClientInstrumentor().instrument()

    # Instrument the FastAPI app.
    # This automatically creates parent spans for requests and propagates the trace context.
    FastAPIInstrumentor.instrument_app(app)

    uvicorn.run(app, host="0.0.0.0", port=8080)
    
    

### Create `pyproject.toml`

In [ ]:
%%writefile deploy/pyproject.toml
[project]
name = "deploy-and-manage-adk-service"
version = "0.1.0"
description = "Add your description here"
readme = "README.md"
requires-python = ">=3.12"
dependencies = [
    "google-adk==1.5.0",
    "locust==2.37.10",
    "pg8000==1.31.2",
    "python-dotenv==1.1.0",
    "toolbox-core==0.2.1",
    "google-genai==1.23.0",
    "opentelemetry-instrumentation-fastapi==0.55b1",
    "opentelemetry-api>=1.20.0",
    "opentelemetry-sdk>=1.20.0",
    "opentelemetry-instrumentation-fastapi>=0.40.0",
    "opentelemetry-exporter-gcp-trace>=1.9.0",
    "opentelemetry-propagator-gcp==1.9.0",
    "opentelemetry-exporter-gcp-monitoring==1.9.0a0",
    "opentelemetry-exporter-otlp",
    "opentelemetry-instrumentation-requests==0.55b1",
    "opentelemetry-instrumentation-aiohttp-client"
]

[dependency-groups]
dev = [
    "pytest==8.4.0",
    "ruff==0.11.13",
]

### Create a Dockerfile

In [ ]:
%%writefile deploy/Dockerfile
FROM python:3.12-slim
RUN pip install --no-cache-dir uv==0.7.13
WORKDIR /app
COPY . .
RUN uv sync
EXPOSE 8080
CMD ["uv", "run", "uvicorn", "server:app", "--host", "0.0.0.0", "--port", "8080"]


### Deploy the ADK Agent to Cloud Run

Since this Cloud Run service indirectly interacts with sensitive databases, we want to be mindful of security best practices during deployment.

Key security configurations:
- `--network={vpc}` / `--subnet={vpc}`: Deploys the service within our private VPC.
- `--service-account=adk-service-account`: Uses a dedicated service account with least-privilege permissions.
- `--no-allow-unauthenticated`: Enforces IAM authentication for all invocations.
- `--ingress=all`: Allows public network ingress this time to allow users to access the web UI. We secure this with IAP. 
- `--allow-unauthenticated`: Disable IAM authentication. We secure the connection with IAP instead.
- We leverage private DNS resolution and Google Private Access (which we setup in notebook 3) for secure communication between the Agent and MCP Toolbox Cloud Run services.
- We use the oauth client ID of the toolbox-service-account service to authenticate calls between the ADK agent and MCP Toolbox.



In [ ]:
# Get the MCP Toolbox endpoint
toolbox_url = ! gcloud run services describe toolbox --region {region} --format 'value(metadata.annotations."run.googleapis.com/urls")'
toolbox_url = json.loads(toolbox_url[0])[0]
print(f"Toolbox Cloud Run endpoint: {toolbox_url}")

# Get the toolbox-service-account oauth client id
toolbox_oauth_id = !gcloud iam service-accounts describe toolbox-service-account@{project_id}.iam.gserviceaccount.com --format='value("oauth2ClientId")'
toolbox_oauth_id = toolbox_oauth_id[0]
print(f"Toolbox OAuth Client ID: {toolbox_oauth_id}")

# Get Cloud SQL connection name
cloud_sql_psc_ip = !gcloud compute forwarding-rules describe psc-forwarding-rule-{cloud_sql_instance} \
    --region={region} \
    --project={project_id} \
    --format="value(IPAddress)"
cloud_sql_psc_ip = cloud_sql_psc_ip[0]

# Construct the SESSION_SERVICE_URI
session_service_uri = f"postgresql+pg8000://postgres:{cloud_sql_password}@{cloud_sql_psc_ip}:5432/postgres"

print(f"SESSION_SERVICE_URI: {session_service_uri}")

!gcloud beta run deploy finance-agent --quiet --no-user-output-enabled \
    --source ./deploy/ \
    --network={vpc} \
    --subnet={vpc} \
    --ingress=all \
    --port 8080 \
    --project {project_id} \
    --region {region} \
    --allow-unauthenticated \
    --update-env-vars=TOOLBOX_URL={toolbox_url},GOOGLE_GENAI_USE_VERTEXAI=True,SESSION_SERVICE_URI={session_service_uri},GOOGLE_CLOUD_PROJECT={project_id},GOOGLE_CLOUD_LOCATION={region},GCS_BUCKET_NAME={gcs_bucket_name},TOOLBOX_OAUTH_CLIENT_ID={toolbox_oauth_id} \
    --min=1 \
    --service-account=adk-service-account \
    --memory=1Gi \
    --iap

## Grant Necessary Service Account Permissions for the ADK Service

Now that we are using both IAP and IAM to protect the toolbox service, we need to grant permissions in 3 places for this to work:
1. IAM `roles/run.invoker` permissions on the toolbox service for the **ADK service account**.
2. IAP `roles/iap.httpsResourceAccessor`role policy permissions on the toolbox service for the **ADK service account**.
3. IAP programmatic access on the project for the oauth client ID of the **service-side service account** (e.g. `toolbox-service-account`). Note that we already configured this permission in the Toolbox deployment section, so we can skip it here.

### Grant `roles/run.invoker` to the ADK Service Account

Since we enabled IAM authentication on the toolbox service as a second layer of security, we will need to grant IAM invoke access to the finance-agent in addition to authenticating the tool invocations with IAP. 

In [ ]:
# Get account finance-agent service account
adk_service_account = !gcloud run services describe finance-agent \
    --region={region} \
    --format="value(spec.template.spec.serviceAccountName)"
adk_service_account = adk_service_account[0]
adk_service_account = f"{adk_service_account}@{project_id}.iam.gserviceaccount.com"
print(f"ADK Service Account: {adk_service_account}")

# Grant service account invoke permissions on toolbox
!gcloud run services add-iam-policy-binding toolbox --region={region} \
    --member=serviceAccount:{adk_service_account} \
    --role="roles/run.invoker"

### Grant `roles/iap.httpsResourceAccessor` to the ADK Service Account

In [ ]:
# Grant httpsResourceAccessor IAP access on toolbox to the notebook service account
!gcloud beta iap web add-iam-policy-binding \
    --member=serviceAccount:{adk_service_account} \
    --role=roles/iap.httpsResourceAccessor \
    --region={region} \
    --resource-type=cloud-run \
    --service=toolbox \
    --condition=None

## Grant Necessary User Permissions

### Trying Accessing the Web UI

1. Run the cell below to get your web UI URL, and try to access it. You'll be prompted to login. 
2. Login with your student account when prompted. You will get an error saying "You don't have access." This is expected. We'll add permissions in the next cell.

In [ ]:
# Retrieve the Web UI URL
agent_url = !gcloud run services describe finance-agent --region {region} --format 'value(status.url)'
agent_url = agent_url[0]
print(agent_url)

### Add IAP User Permissions

Next, we'll add permissions for your student account so that you can securely access the Web UI. Replace `student_email` with the email address you are using to access the Google Cloud Console.

In [ ]:
# Replace the variable value for student_email below with your student email address.
student_email = "user@example.com"

# Add IAP access to the ADK Cloud Run service for your student account
!gcloud beta iap web add-iam-policy-binding \
    --member=user:{student_email} \
    --role=roles/iap.httpsResourceAccessor \
    --region={region} \
    --resource-type=cloud-run \
    --service=finance-agent \
    --condition=None

print(f"{agent_url}?gcp-iap-mode=CLEAR_LOGIN_COOKIE")

### Access the UI

Now wait for a little over a minute and try access the UI again. This time you should see an interface like the screenshot below.

![ADK UI](img/adk-ui/adk-ui.png)

## Test the Agent

Now you can interact with your agent through the web UI. The following scenarios are designed to test the different tools you have configured. Pay attention to how the agent selects and uses the appropriate tool based on your natural language questions. You can also click on the tool calls in the UI to see the detailed execution flow.

> NOTE: If you see errors like the following, check your permissions in the cells above and try again: `{"error": "401, message='Attempt to decode JSON with unexpected mimetype: text/html; charset=utf-8', url='https://toolbox-123456789098.us-central1.run.app/api/tool/get_loan_repayments/invoke'"}`

### Scenario 1: Investigating a Customer's Loan Repayments
This conversation flow utilizes the Spanner `get_loan_repayments` tool to investigate a customer's financial history.

- Initial User Question: "Can you show me the loan repayment history for customer ID 263?"
- Follow-up Question 1: "Thanks. Can you tell me the total amount of the loans this customer has repaid?"
- Follow-up Question 2: "Which account was used for the most recent repayment, and what was the date of that transaction?"
- Follow-up Question 3: "Based on the data, what is the largest single loan amount this customer has taken out?"

Click on the `get_loan_repayments` tool call to see a graph of the conversation execution flow and details about each step.

![ADK Scenario 1](img/adk-ui/scenario_1.png)


### Scenario 2: Analyzing Suspicious Account Activity
This exchange uses the Spanner `get_account_inflows_outflows` tool to analyze the movement of money.

- Initial User Question: "I need to analyze the account activity for customer 330. Can you get the total inflows and outflows for their accounts?"
- Follow-up Question 1: "Which of their accounts has the highest ratio of inflows to outflows?"
- Follow-up Question 2: "Is the account with the high inflow/outflow ratio currently blocked? And what type of account is it?"
- Follow-up Question 3: "Can you list all the accounts for this customer and their corresponding money flow ratios?"

Click on the `get_account_inflows_outflows` tool call to see a graph of the conversation execution flow and details about each step.

![ADK Scenario 2](img/adk-ui/scenario_2.png)

### Scenario 3: Tracing Potential Money Laundering
This conversation uses the Spanner  `get_indirect_transfers` and `get_destination_account_audit_events` tools to trace a series of transactions, and it uses the `block_account` tool to take action on the investigation.

- Initial User Question: "I suspect there might be some indirect money transfers between account 75 and account 199. Can you check if there were any indirect transfers between them?"
- Follow-up Question 1: "How many hops were there in the transfer? Also, can you tell me if either the source or destination account is currently blocked?"
- Follow-up Question 2: "There's some activity on a related account, account 11, that seems suspicious. Can you pull the latest audit events for accounts that received a transfer from that account?"
- Follow-up Question 3: "From the audit trail, can you identify the timestamps of the audit events and any details associated with them? Flag anything that seems suspicious."
- Follow-up Action 1: "Block account 446."
- Follow-up Question 4: "Who owns the intermediary accounts for the indirect transfers?"
- Follow-up Action 2: "Block all the intermediary accounts."


Click on the `get_loan_repayments` tool call to see a graph of the conversation execution flow and details about each step.

![ADK Scenario 3](img/adk-ui/scenario_3.png)

### Scenario 4: General Database Queries
This conversation flow demonstrates the use of the flexible `get_finance_database_context` tool for more open-ended questions about the finance database in AlloyDB.

- Initial User Question: "How many customers do we have?"
- Follow-up Question 1: "Can you give me a count of all transactions that have been flagged for fraud?"
- Follow-up Question 2: "What are the top 5 merchant categories that are flagged for fraud?"
- Follow-up Question 3: "Compare that with the list of the top 5 merchant categories by transaction volume."

Click on the `get_loan_repayments` tool call to see a graph of the conversation execution flow and details about each step.

![ADK Scenario 4](img/adk-ui/scenario_4.png)

Congratulations, you have successfully completed the course! By completing this course, you have successfully designed, built, and deployed a sophisticated and secure AI agent capable of interacting with sensitive financial data in an enterprise environment. You've gained hands-on experience with Google's Agent Development Kit (ADK) and MCP Toolbox for Databases, learning how to create and customize powerful tools for your agent. More importantly, you've mastered the critical security principles and services necessary for enterprise-grade AI, including the configuration of private networks (VPC, PSC, PSA, GPA) and the implementation of robust access controls with Identity-Aware Proxy (IAP). You are now equipped with the knowledge to build and deploy your own secure, interactive, and autonomous agents that can safely connect to and reason about sensitive data in a real-world enterprise setting.